# Importando as Bibliotecas

In [1]:
import ranking
import salvar_resultados as sr

import pandas as pd
import numpy as np
import string

import time

# Constantes

In [2]:
COLUNAS = ("indice", "titulo_sa", "ean", "categoria")
LISTA_CATEGORIAS = ("celulares", "notebooks", "geladeiras", "fogoes", "tvs")

# Lendo os Datasets

In [ ]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

# Funções

In [ ]:
def criar_dicionario(indice, titulo_sa, ean, categoria):
    
    return {
            'indice' : indice, 'titulo_sa' : titulo_sa, 'ean' : ean, 'categoria' : categoria
           }


def criar_df_match(df_concat, ean_repetido):

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo_sa = df_concat.loc[indice]['titulo_sa'],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria']
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


def salvar_resultado(nome, df):

    df.to_csv(f'Dados/Resultados/Cos_Rankeado/{nome}_métricas.csv', index = False)


def salvar_resultado_categoria(nome, df):

    for categoria in LISTA_CATEGORIAS:

        df[df["categoria"] == categoria].to_csv(f'Dados/Resultados/Cos_Rankeado/{categoria}/{nome}_métricas.csv', index = False)

# BoW

In [ ]:
flag_cos = True
if flag_cos == True:

    dir_metricas = f"Dados/Resultados/Ranqueado/Cos"

    for nome, df_teste in zip(arquivos, lista_df_teste):

        # remoção de pontuação e acentos
        ranking.fazer_pre_processamento(df_teste)

        df_concat, df_matches = ranking.criar_df_teste(df_teste)

        # calculando a quantidade máxima de palavras no título
        tam_max = ranking.calcular_tam_max(df_concat['titulo_sa'])

        inicio_tempo = time.time()
        # calculando o BoW de cada título
        cv, titulo_bow = ranking.formatar_entrada_bow(df_concat['titulo_sa'], mf = tam_max)
        # calculando a distância entre os vetores (apenas linhas com match)
        resultado = ranking.calcular_dis_2_vetores_cond(titulo_bow, df_matches)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

        break

In [ ]:
    a

# Resultados

In [3]:
def print_acu_k(df, flag = True):

    linha_1 = f"  | k-1\t | k-10\t| k-50\t|"

    if flag == True:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]}\t | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    else:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]} | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    linha_3 = f"0 | {df['k-1'].value_counts()[0]}\t | {df['k-10'].value_counts()[0]}\t| {df['k-50'].value_counts()[0]}\t|"

    print(f"{linha_1}\n{linha_2}\n{linha_3}")


def print_describe(df):

    print("")
    print(df[["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe())

## Geral

In [4]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_resultado = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/{arquivo}_métricas.csv")
    lista_df_resultado.append(df_treino)

### Aleátorio

#### 1:1

In [5]:
df_ale_1_1 = lista_df_resultado[0]
print_acu_k(df_ale_1_1)
print_describe(df_ale_1_1)

  | k-1	 | k-10	| k-50	|
1 | 200	 | 367	| 438	|
0 | 258	 | 91	| 20	|

              k-1       k-10        k-50  match_rank  1/match_rank
count  458.000000  458.00000  458.000000  458.000000    458.000000
mean     0.436681    0.80131    0.956332   11.093886      0.556268
std      0.496517    0.39945    0.204579   29.148565      0.412447
min      0.000000    0.00000    0.000000    1.000000      0.003704
25%      0.000000    1.00000    1.000000    1.000000      0.142857
50%      0.000000    1.00000    1.000000    2.000000      0.500000
75%      1.000000    1.00000    1.000000    7.000000      1.000000
max      1.000000    1.00000    1.000000  270.000000      1.000000


#### 5:1

In [6]:
df_ale_5_1 = lista_df_resultado[1]
print_acu_k(df_ale_5_1, False)
print_describe(df_ale_5_1)

  | k-1	 | k-10	| k-50	|
1 | 225 | 474	| 581	|
0 | 406	 | 157	| 50	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  631.000000  631.000000  631.000000  631.000000    631.000000
mean     0.356577    0.751189    0.920761   15.705230      0.485451
std      0.479368    0.432667    0.270326   38.368124      0.407919
min      0.000000    0.000000    0.000000    1.000000      0.002283
25%      0.000000    1.000000    1.000000    1.000000      0.100000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   10.000000      1.000000
max      1.000000    1.000000    1.000000  438.000000      1.000000


### Hard Negative

#### 1:1

In [12]:
df_hn_1_1 = lista_df_resultado[2]
print_acu_k(df_hn_1_1)
print_describe(df_hn_1_1)

  | k-1	 | k-10	| k-50	|
1 | 192	 | 422	| 499	|
0 | 338	 | 108	| 31	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  530.000000  530.000000  530.000000  530.000000    530.000000
mean     0.362264    0.796226    0.941509   11.864151      0.504844
std      0.481109    0.403183    0.234890   32.191835      0.398234
min      0.000000    0.000000    0.000000    1.000000      0.002625
25%      0.000000    1.000000    1.000000    1.000000      0.125000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000    8.000000      1.000000
max      1.000000    1.000000    1.000000  381.000000      1.000000


#### 5:1

In [13]:
df_hn_5_1 = lista_df_resultado[3]
print_acu_k(df_hn_5_1, False)
print_describe(df_hn_5_1)

  | k-1	 | k-10	| k-50	|
1 | 266 | 574	| 694	|
0 | 490	 | 182	| 62	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  756.000000  756.000000  756.000000  756.000000    756.000000
mean     0.351852    0.759259    0.917989   16.727513      0.483830
std      0.477864    0.427816    0.274562   47.739468      0.405226
min      0.000000    0.000000    0.000000    1.000000      0.001645
25%      0.000000    1.000000    1.000000    1.000000      0.100000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   10.000000      1.000000
max      1.000000    1.000000    1.000000  608.000000      1.000000


## Por Categoria

### Celulares

In [9]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_celulares = []
for arquivo in arquivos:

    df_celulares = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/celulares/{arquivo}_métricas.csv")
    lista_df_celulares.append(df_celulares)

In [10]:
df_ale_1_1_cel = lista_df_celulares[0]
df_ale_5_1_cel = lista_df_celulares[1]
df_hn_1_1_cel = lista_df_celulares[2]
df_hn_5_1_cel = lista_df_celulares[3]

In [11]:
print_acu_k(df_ale_1_1_cel)
print_describe(df_ale_1_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 34	 | 67	| 80	|
0 | 49	 | 16	| 3	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  83.000000  83.000000  83.000000   83.000000     83.000000
mean    0.409639   0.807229   0.963855   11.253012      0.516927
std     0.494757   0.396873   0.187784   31.363270      0.419709
min     0.000000   0.000000   0.000000    1.000000      0.003704
25%     0.000000   1.000000   1.000000    1.000000      0.142857
50%     0.000000   1.000000   1.000000    3.000000      0.333333
75%     1.000000   1.000000   1.000000    7.000000      1.000000
max     1.000000   1.000000   1.000000  270.000000      1.000000


In [12]:
print_acu_k(df_ale_5_1_cel)
print_describe(df_ale_5_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 24	 | 62	| 76	|
0 | 64	 | 26	| 12	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  88.000000  88.000000  88.000000   88.000000     88.000000
mean    0.272727   0.704545   0.863636   20.011364      0.418079
std     0.447914   0.458861   0.345141   37.847393      0.391428
min     0.000000   0.000000   0.000000    1.000000      0.004808
25%     0.000000   0.000000   1.000000    1.000000      0.071429
50%     0.000000   1.000000   1.000000    4.000000      0.250000
75%     1.000000   1.000000   1.000000   14.000000      1.000000
max     1.000000   1.000000   1.000000  208.000000      1.000000


In [13]:
print_acu_k(df_hn_1_1_cel)
print_describe(df_hn_1_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 26	 | 60	| 76	|
0 | 57	 | 23	| 7	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  83.000000  83.000000  83.000000   83.000000     83.000000
mean    0.313253   0.722892   0.915663   13.012048      0.461356
std     0.466636   0.450291   0.279582   25.840738      0.393800
min     0.000000   0.000000   0.000000    1.000000      0.005682
25%     0.000000   0.000000   1.000000    1.000000      0.077381
50%     0.000000   1.000000   1.000000    3.000000      0.333333
75%     1.000000   1.000000   1.000000   13.000000      1.000000
max     1.000000   1.000000   1.000000  176.000000      1.000000


In [14]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_cel['k-1'].value_counts()[1]} | {df_hn_5_1_cel['k-10'].value_counts()[1]}\t| {df_hn_5_1_cel['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_cel['k-1'].value_counts()[0]}\t | {df_hn_5_1_cel['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_cel)

  | k-1	 | k-10	| k-50	|
1 | 38 | 84	| 106	|
0 | 83	 | 37	|	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  121.000000  121.000000  121.000000  121.000000    121.000000
mean     0.314050    0.694215    0.876033   19.074380      0.430891
std      0.466066    0.462655    0.330914   36.699265      0.409205
min      0.000000    0.000000    0.000000    1.000000      0.003984
25%      0.000000    0.000000    1.000000    1.000000      0.066667
50%      0.000000    1.000000    1.000000    4.000000      0.250000
75%      1.000000    1.000000    1.000000   15.000000      1.000000
max      1.000000    1.000000    1.000000  251.000000      1.000000


### Fogões

In [16]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_fogoes = []
for arquivo in arquivos:

    df_fogoes = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/fogoes/{arquivo}_métricas.csv")
    lista_df_fogoes.append(df_fogoes)

In [17]:
df_ale_1_1_fg = lista_df_fogoes[0]
df_ale_5_1_fg = lista_df_fogoes[1]
df_hn_1_1_fg = lista_df_fogoes[2]
df_hn_5_1_fg = lista_df_fogoes[3]

In [18]:
print_acu_k(df_ale_1_1_fg)
print_describe(df_ale_1_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 48	 | 112	| 135	|
0 | 100	 | 36	| 13	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  148.000000  148.000000  148.000000  148.000000    148.000000
mean     0.324324    0.756757    0.912162   16.783784      0.467728
std      0.469711    0.430498    0.284020   40.441196      0.396996
min      0.000000    0.000000    0.000000    1.000000      0.003891
25%      0.000000    1.000000    1.000000    1.000000      0.100000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   10.000000      1.000000
max      1.000000    1.000000    1.000000  257.000000      1.000000


In [19]:
print_acu_k(df_ale_5_1_fg)
print_describe(df_ale_5_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 52	 | 156	| 200	|
0 | 168	 | 64	| 20	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  220.000000  220.000000  220.000000  220.000000    220.000000
mean     0.236364    0.709091    0.909091   17.900000      0.395824
std      0.425817    0.455217    0.288135   38.251976      0.372481
min      0.000000    0.000000    0.000000    1.000000      0.003436
25%      0.000000    0.000000    1.000000    2.000000      0.083333
50%      0.000000    1.000000    1.000000    4.000000      0.250000
75%      0.000000    1.000000    1.000000   12.000000      0.500000
max      1.000000    1.000000    1.000000  291.000000      1.000000


In [20]:
print_acu_k(df_hn_1_1_fg)
print_describe(df_hn_1_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 41	 | 132	| 162	|
0 | 135	 | 44	| 14	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  176.000000  176.000000  176.000000  176.000000    176.000000
mean     0.232955    0.750000    0.920455   16.511364      0.391153
std      0.423920    0.434248    0.271360   43.083837      0.367786
min      0.000000    0.000000    0.000000    1.000000      0.002625
25%      0.000000    0.750000    1.000000    2.000000      0.097727
50%      0.000000    1.000000    1.000000    4.000000      0.250000
75%      0.000000    1.000000    1.000000   10.250000      0.500000
max      1.000000    1.000000    1.000000  381.000000      1.000000


In [21]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_fg['k-1'].value_counts()[1]}  | {df_hn_5_1_fg['k-10'].value_counts()[1]}\t| {df_hn_5_1_fg['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_fg['k-1'].value_counts()[0]}\t | {df_hn_5_1_fg['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_fg)

  | k-1	 | k-10	| k-50	|
1 | 56  | 163	| 206	|
0 | 172	 | 65	|	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  228.000000  228.000000  228.000000  228.000000    228.000000
mean     0.245614    0.714912    0.903509   21.855263      0.398620
std      0.431398    0.452450    0.295913   64.267414      0.375013
min      0.000000    0.000000    0.000000    1.000000      0.001645
25%      0.000000    0.000000    1.000000    2.000000      0.075549
50%      0.000000    1.000000    1.000000    4.000000      0.250000
75%      0.000000    1.000000    1.000000   13.250000      0.500000
max      1.000000    1.000000    1.000000  608.000000      1.000000


### Geladeiras

In [22]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_gel = []
for arquivo in arquivos:

    df_gel = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/geladeiras/{arquivo}_métricas.csv")
    lista_df_gel.append(df_gel)

In [23]:
df_ale_1_1_gel = lista_df_gel[0]
df_ale_5_1_gel = lista_df_gel[1]
df_hn_1_1_gel = lista_df_gel[2]
df_hn_5_1_gel = lista_df_gel[3]

In [24]:
print_acu_k(df_ale_1_1_gel)
print_describe(df_ale_1_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 32	 | 54	| 66	|
0 | 35	 | 13	| 1	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  67.000000  67.000000  67.000000   67.000000     67.000000
mean    0.477612   0.805970   0.985075    7.119403      0.608131
std     0.503268   0.398437   0.122169   12.628158      0.402144
min     0.000000   0.000000   0.000000    1.000000      0.014286
25%     0.000000   1.000000   1.000000    1.000000      0.250000
50%     0.000000   1.000000   1.000000    2.000000      0.500000
75%     1.000000   1.000000   1.000000    4.000000      1.000000
max     1.000000   1.000000   1.000000   70.000000      1.000000


In [25]:
print_acu_k(df_ale_5_1_gel)
print_describe(df_ale_5_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 33	 | 72	| 91	|
0 | 63	 | 24	| 5	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  96.000000  96.000000  96.000000   96.000000     96.000000
mean    0.343750   0.750000   0.947917   13.135417      0.472809
std     0.477452   0.435286   0.223361   27.120247      0.406697
min     0.000000   0.000000   0.000000    1.000000      0.006410
25%     0.000000   0.750000   1.000000    1.000000      0.095833
50%     0.000000   1.000000   1.000000    3.000000      0.333333
75%     1.000000   1.000000   1.000000   10.500000      1.000000
max     1.000000   1.000000   1.000000  156.000000      1.000000


In [26]:
print_acu_k(df_hn_1_1_gel)
print_describe(df_hn_1_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 22	 | 61	| 68	|
0 | 49	 | 10	| 3	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  71.000000  71.000000  71.000000   71.000000     71.000000
mean    0.309859   0.859155   0.957746    7.380282      0.513031
std     0.465727   0.350338   0.202599   14.733214      0.359136
min     0.000000   0.000000   0.000000    1.000000      0.014085
25%     0.000000   1.000000   1.000000    1.000000      0.250000
50%     0.000000   1.000000   1.000000    2.000000      0.500000
75%     1.000000   1.000000   1.000000    4.000000      1.000000
max     1.000000   1.000000   1.000000   71.000000      1.000000


In [27]:
print_acu_k(df_hn_5_1_gel)
print_describe(df_hn_5_1_gel)

  | k-1	 | k-10	| k-50	|
1 | 28	 | 81	| 99	|
0 | 80	 | 27	| 9	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  108.000000  108.000000  108.000000  108.000000    108.000000
mean     0.259259    0.750000    0.916667   15.407407      0.419145
std      0.440271    0.435031    0.277674   33.950221      0.376393
min      0.000000    0.000000    0.000000    1.000000      0.003906
25%      0.000000    0.750000    1.000000    1.000000      0.097727
50%      0.000000    1.000000    1.000000    3.500000      0.291667
75%      1.000000    1.000000    1.000000   10.250000      1.000000
max      1.000000    1.000000    1.000000  256.000000      1.000000


### Notebooks

In [28]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_not = []
for arquivo in arquivos:

    df_not = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/notebooks/{arquivo}_métricas.csv")
    lista_df_not.append(df_not)

In [29]:
df_ale_1_1_not = lista_df_not[0]
df_ale_5_1_not = lista_df_not[1]
df_hn_1_1_not = lista_df_not[2]
df_hn_5_1_not = lista_df_not[3]

In [30]:
print_acu_k(df_ale_1_1_not)
print_describe(df_ale_1_1_not)

  | k-1	 | k-10	| k-50	|
1 | 55	 | 76	| 85	|
0 | 33	 | 12	| 3	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  88.000000  88.000000  88.000000   88.000000     88.000000
mean    0.625000   0.863636   0.965909    7.431818      0.713611
std     0.486897   0.345141   0.182503   21.731801      0.390252
min     0.000000   0.000000   0.000000    1.000000      0.006849
25%     0.000000   1.000000   1.000000    1.000000      0.333333
50%     1.000000   1.000000   1.000000    1.000000      1.000000
75%     1.000000   1.000000   1.000000    3.000000      1.000000
max     1.000000   1.000000   1.000000  146.000000      1.000000


In [31]:
print_acu_k(df_ale_5_1_not)
print_describe(df_ale_5_1_not)

  | k-1	 | k-10	| k-50	|
1 | 64	 | 97	| 104	|
0 | 45	 | 12	| 5	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  109.000000  109.000000  109.000000  109.000000    109.000000
mean     0.587156    0.889908    0.954128   11.899083      0.675610
std      0.494619    0.314450    0.210173   51.093767      0.400808
min      0.000000    0.000000    0.000000    1.000000      0.002283
25%      0.000000    1.000000    1.000000    1.000000      0.250000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    4.000000      1.000000
max      1.000000    1.000000    1.000000  438.000000      1.000000


In [32]:
print_acu_k(df_hn_1_1_not)
print_describe(df_hn_1_1_not)

  | k-1	 | k-10	| k-50	|
1 | 68	 | 101	| 112	|
0 | 48	 | 15	| 4	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  116.000000  116.000000  116.000000  116.000000    116.000000
mean     0.586207    0.870690    0.965517    7.646552      0.671636
std      0.494649    0.336999    0.183257   21.402663      0.406003
min      0.000000    0.000000    0.000000    1.000000      0.006494
25%      0.000000    1.000000    1.000000    1.000000      0.200000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    5.000000      1.000000
max      1.000000    1.000000    1.000000  154.000000      1.000000


In [33]:
print_acu_k(df_hn_5_1_not)
print_describe(df_hn_5_1_not)

  | k-1	 | k-10	| k-50	|
1 | 96	 | 152	| 165	|
0 | 74	 | 18	| 5	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  170.000000  170.000000  170.000000  170.000000    170.000000
mean     0.564706    0.894118    0.970588   10.623529      0.669822
std      0.497260    0.308596    0.169457   46.761760      0.394970
min      0.000000    0.000000    0.000000    1.000000      0.002304
25%      0.000000    1.000000    1.000000    1.000000      0.250000
50%      1.000000    1.000000    1.000000    1.000000      1.000000
75%      1.000000    1.000000    1.000000    4.000000      1.000000
max      1.000000    1.000000    1.000000  434.000000      1.000000


### TVs

In [34]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_tv = []
for arquivo in arquivos:

    df_tv = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/tvs/{arquivo}_métricas.csv")
    lista_df_tv.append(df_tv)

In [35]:
df_ale_1_1_tv = lista_df_tv[0]
df_ale_5_1_tv = lista_df_tv[1]
df_hn_1_1_tv = lista_df_tv[2]
df_hn_5_1_tv = lista_df_tv[3]

In [36]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_ale_1_1_tv['k-1'].value_counts()[1]}   | {df_ale_1_1_tv['k-10'].value_counts()[1]}\t| {df_ale_1_1_tv['k-50'].value_counts()[1]}\t|\n0 | {df_ale_1_1_tv['k-1'].value_counts()[0]}\t | {df_ale_1_1_tv['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_ale_1_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 31   | 58	| 72	|
0 | 41	 | 14	|	|

             k-1       k-10  k-50  match_rank  1/match_rank
count  72.000000  72.000000  72.0   72.000000     72.000000
mean    0.430556   0.805556   1.0    7.388889      0.543046
std     0.498629   0.398550   0.0   10.329168      0.420350
min     0.000000   0.000000   1.0    1.000000      0.022727
25%     0.000000   1.000000   1.0    1.000000      0.111111
50%     0.000000   1.000000   1.0    2.000000      0.500000
75%     1.000000   1.000000   1.0    9.000000      1.000000
max     1.000000   1.000000   1.0   44.000000      1.000000


In [37]:
print_acu_k(df_ale_5_1_tv)
print_describe(df_ale_5_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 52	 | 87	| 110	|
0 | 66	 | 31	| 8	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  118.000000  118.000000  118.000000  118.000000    118.000000
mean     0.440678    0.737288    0.932203   14.008475      0.537428
std      0.498586    0.441984    0.252469   32.726135      0.429781
min      0.000000    0.000000    0.000000    1.000000      0.004975
25%      0.000000    0.000000    1.000000    1.000000      0.090909
50%      0.000000    1.000000    1.000000    2.000000      0.500000
75%      1.000000    1.000000    1.000000   11.000000      1.000000
max      1.000000    1.000000    1.000000  201.000000      1.000000


In [38]:
print_acu_k(df_hn_1_1_tv)
print_describe(df_hn_1_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 35	 | 68	| 81	|
0 | 49	 | 16	| 3	|

             k-1       k-10       k-50  match_rank  1/match_rank
count  84.000000  84.000000  84.000000   84.000000     84.000000
mean    0.416667   0.809524   0.964286   10.607143      0.548772
std     0.495968   0.395035   0.186691   33.511378      0.406212
min     0.000000   0.000000   0.000000    1.000000      0.003484
25%     0.000000   1.000000   1.000000    1.000000      0.142857
50%     0.000000   1.000000   1.000000    2.000000      0.500000
75%     1.000000   1.000000   1.000000    7.000000      1.000000
max     1.000000   1.000000   1.000000  287.000000      1.000000


In [39]:
print_acu_k(df_hn_5_1_tv)
print_describe(df_hn_5_1_tv)

  | k-1	 | k-10	| k-50	|
1 | 48	 | 94	| 118	|
0 | 81	 | 35	| 11	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  129.000000  129.000000  129.000000  129.000000    129.000000
mean     0.372093    0.728682    0.914729   14.612403      0.493139
std      0.485247    0.446373    0.280374   29.954734      0.415499
min      0.000000    0.000000    0.000000    1.000000      0.005556
25%      0.000000    0.000000    1.000000    1.000000      0.083333
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   12.000000      1.000000
max      1.000000    1.000000    1.000000  180.000000      1.000000
